In [191]:
import pandas as pd
from os import listdir
import unidecode

In [192]:
additional_info = pd.read_csv('additional_csv/addinfo.csv')
renta_pv = pd.read_csv('additional_csv/rentas_PV.csv',  encoding='ISO-8859-1', sep=';')
rentas_espanya = pd.read_csv('additional_csv/rentas_espanya.csv').rename(columns={'Unnamed: 0':'provincia'})

In [193]:
# Remove code, accencts and all names to lower case
additional_info['poblacion'] = additional_info['poblacion'].apply(lambda x: unidecode.unidecode(' '.join(x.split(' ')[1:])).lower())

rentas_espanya['provincia'] = rentas_espanya['provincia'].apply(lambda x: unidecode.unidecode(''.join(x.split('-')[0])).lower())

renta_pv['ambitos territoriales'] = renta_pv['ambitos territoriales'].apply(lambda x: unidecode.unidecode(x).lower())

In [194]:
all_files = []
path = 'province_houses/'
for filename in listdir(path):
    if 'houses' in filename and 'csv' in filename:
        provincia = unidecode.unidecode(' '.join(filename.split('_')[1:len(filename.split('_')):1]).split('.')[0])
        renta_media = rentas_espanya[rentas_espanya['provincia'].str.contains(provincia)]['Renta disponible media']
        if renta_media.empty:
            renta_media = renta_pv[(renta_pv['ambitos territoriales'].str.contains(provincia)) & (renta_pv['tipo de renta'] == 'Renta total')]['2016']
        if renta_media.empty:
            renta_media = None
        else:
            renta_media = renta_media.values[0]
            
        valor_empresa_muni = pd.to_numeric(additional_info[(additional_info['poblacion'].str.contains(provincia)) & (additional_info['source'] == 'Empresas')]['valor'].values[0].replace('.', ''))
        valor_poblacion_muni = pd.to_numeric(additional_info[(additional_info['poblacion'].str.contains(provincia)) & (additional_info['source'] == 'Poblacion')]['valor'].values[0].replace('.', ''))
        
        valor_empresa_nacional = pd.to_numeric(additional_info[(additional_info['poblacion'].str.contains('nacional')) & (additional_info['source'] == 'Empresas')]['valor'].values[0].replace('.', ''))
        valor_poblacion_nacional = pd.to_numeric(additional_info[(additional_info['poblacion'].str.contains('nacional')) & (additional_info['source'] == 'Poblacion')]['valor'].values[0].replace('.', ''))

        
        df = pd.read_csv(path + filename)
        df['source'] = filename.split('.')[0]
        df['number_of_companies_prov'] = valor_empresa_muni
        df['population_prov'] = valor_poblacion_muni
        df['companies_prov_vs_national_%'] = ((valor_empresa_muni / valor_empresa_nacional) * 100).round(2)
        df['population_prov_vs_national_%'] = ((valor_poblacion_muni / valor_poblacion_nacional) * 100).round(2)
        df['renta_media_prov'] = renta_media

        
        all_files.append(df)

In [195]:
result = pd.concat(all_files)

In [196]:
result.to_csv('spanish_houses.csv', index=False)